Top2Vec 是一种用于主题建模和语义搜索的算法。**我个人从理解代码和使用代码难度来看， 对于Python小白，BERTopic更适合直接用预训练词向量，而Top2Vec更适合对小规模数据训练词向量后做主题建模。**它自动检测文本中存在的主题并生成联合嵌入的主题、文档和词向量。训练 Top2Vec 模型后，您可以：

- 获取检测到的主题数。
- 获取话题。
- 获取主题大小。
- 获取分层主题。
- 按关键字搜索主题。
- 按主题搜索文档。
- 按关键字搜索文档。
- 找出相似的词。
- 查找类似的文档。
- 使用 RESTful-Top2Vec 公开模型
- 有关其工作原理的更多详细信息，请参阅论文。



**亮点**

- 自动查找主题数。
- 不需要停用词列表。
- 不需要词干/词形还原。
- 适用于短文本。
- 创建联合嵌入的主题、文档和词向量。
- 内置搜索功能。


**它是如何工作的？**

该算法做出的假设是，许多语义相似的文档都表明了一个潜在的主题。

第一步是创建文档和词向量的联合嵌入。一旦文档和单词被嵌入到一个向量空间中，算法的目标就是找到密集的文档集群，然后确定哪些单词将这些文档吸引到一起。每个密集区域是一个主题，将文档吸引到密集区域的词就是主题词。



<br>

In [ ]:
!pip3 install top2vec==1.0.27

## 1. 导入数据

使用某灾难数据集，这里是存在标注的标签，但是我们假设不用label的，仅作为评判Top2vec运行效果的标准。

In [2]:
from IPython.display import display
import cntext as ct
from top2vec import Top2Vec
import pandas as pd
import jieba

stopwords = ct.load_pkl_dict('STOPWORDS.pkl')['STOPWORDS']['chinese']

df = pd.read_csv('cnews.csv')
df.head()

label                                               text
0    教育  澳移民子女成长记：带着中国心融入主流社会新华网悉尼5月31日电 无论哪个国家的父辈与子女间都...
1    体育  快船vs火箭首发：休城旨在练兵 小德帕特森进先发新浪体育讯北京时间4月10日消息，在常规赛还...
2    科技  3英寸屏高清闪存DV 三洋TH1特价1499         作者：中关村在线 飘雪    ...
3    时尚  贝嫂乏味归乏味 还有人买账Victoria Beckham 乏味归乏味 还有人买账大姐大的阵...
4    房产  三亚岭南赶房集 金九银十再兴购房游纯粹的旅行闲适有余却“收获”不足，设计一条可以兼容曼妙风景...

In [3]:
df.label.value_counts()

时政    120
科技    106
时尚    106
财经    105
家居    103
教育     97
娱乐     96
体育     95
房产     87
游戏     85
Name: label, dtype: int64

## 2. 清洗数据

一般而言，作中文文本分析，需要把中文分词构造成类西方语言(空格间隔词语的文本)风格。在此期间，顺便将停用词剔除。其实在用top2vec时，不剔除停用词影响也不大。


In [5]:
def clean_text(text):
    words = jieba.lcut(text)
    words = [w for w in words if w not in stopwords]
    return ' '.join(words)


df['cleantext'] = df.text.apply(clean_text)
df.head()

label                                               text  \
0    教育  澳移民子女成长记：带着中国心融入主流社会新华网悉尼5月31日电 无论哪个国家的父辈与子女间都...   
1    体育  快船vs火箭首发：休城旨在练兵 小德帕特森进先发新浪体育讯北京时间4月10日消息，在常规赛还...   
2    科技  3英寸屏高清闪存DV 三洋TH1特价1499         作者：中关村在线 飘雪    ...   
3    时尚  贝嫂乏味归乏味 还有人买账Victoria Beckham 乏味归乏味 还有人买账大姐大的阵...   
4    房产  三亚岭南赶房集 金九银十再兴购房游纯粹的旅行闲适有余却“收获”不足，设计一条可以兼容曼妙风景...   

                                           cleantext  
0  澳 移民 子女 成长 记 带 中国 心 融入 主流 社会 新华网 悉尼 5 31 日电   ...  
1  快船 vs 火箭 首发 休城 旨在 练兵   小德 帕特森 进先发 新浪 体育讯 北京 时间...  
2  3 英寸 屏 高清 闪存 DV   三洋 TH1 特价 1499              ...  
3  贝嫂 乏味 乏味   买账 Victoria   Beckham   乏味 乏味   买账 ...  
4  三亚 岭南 赶房集   金九银 十再兴 购房 游 纯粹 旅行 闲适 有余 “ 收获 ” 不足...

## 3. 训练模型

Top2vec有一下四个常用参数

**Top2vec(documents, min_count, speed, workers)**

- documents: 文档列表
- min_count: 词语最少出现次数。低于min_count的词不加入模型中
- speed: 训练速度，参数默认"learn"
  - "fast-learn"  速度最快，训练效果最差
  - "learn"       速度，训练效果中等
  - "deep-learn"  速度最慢，训练效果最佳
- workers: 并行运行数，该值最大取值为电脑CPU的核数。

In [6]:
model = Top2Vec(documents=df['cleantext'].to_list(), 
                min_count=10,
                speed="deep-learn",  
                workers=8)

2022-05-13 19:14:58,896 - top2vec - INFO - Pre-processing documents for training
2022-05-13 19:14:59,343 - top2vec - INFO - Creating joint document/word embedding
2022-05-13 19:18:20,984 - top2vec - INFO - Creating lower dimension embedding of documents
OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
2022-05-13 19:18:29,918 - top2vec - INFO - Finding dense areas of documents
2022-05-13 19:18:29,936 - top2vec - INFO - Finding topics


In [7]:
# 话题个数
model.get_num_topics()

9

In [8]:
# 各话题数量
topic_sizes, topic_nums = model.get_topic_sizes()

{"topic_sizes":topic_sizes, 
 "topic_ids":topic_nums}

{'topic_sizes': array([361, 116, 107,  99,  97,  93,  82,  25,  20]),
 'topic_ids': array([0, 1, 2, 3, 4, 5, 6, 7, 8])}

## 4. get_topics

用pyecharts词云图显示**话题信息**， 为了简化代码，将该功能封装为函数

In [9]:
def gen_wordcloud(topic_words, word_scores, topic_id):
    """
    topic_words: 主题词列表
    word_scores: 主题特征词的权重得分(词语表征主题的能力)
    topic_id: 主题id
    """
    import pyecharts.options as opts
    from pyecharts.charts import WordCloud
    from IPython.display import display
    
    data = [(word, str(weight)) for word, weight in zip(topic_words, word_scores)]

    wc = WordCloud()
    wc.add(series_name="", data_pair=data, word_size_range=[6, 88])
    wc.set_global_opts(
        title_opts=opts.TitleOpts(title="Topic_{topic_id}".format(topic_id=topic_id), 
                                  title_textstyle_opts=opts.TextStyleOpts(font_size=23)),
        tooltip_opts=opts.TooltipOpts(is_show=True))

    display(wc.render_notebook())

In [11]:
topic_wordss, word_scoress, topic_ids = model.get_topics(9)

for topic_words, word_scores, topic_id in zip(topic_wordss, word_scoress, topic_ids):
    gen_wordcloud(topic_words, word_scores, topic_id)

## 5. get_documents_topics

get_documents_topics(doc_ids, num_topics=1)

- doc_ids: 待查询文档id列表
- num_topics: 返回某文档可能归属话题的个数

In [13]:
# 查第一条文档的
model.get_documents_topics(doc_ids=[0], num_topics=1)

(array([0]),
 array([0.1384481], dtype=float32),
 array([['政府', '经济', '政策', '建设', '中方', '发展', '促进', '部门', '留学', '学生', '会议',
         '我要', '事务', '日电', '房价', '教育', '国务院', '温家宝', '留学生', '人数', '移民',
         '会见', '推动', '申请者', '申请', '官员', '住房', '房屋', '加强', '中国政府', '购房',
         '国家', '支付', '楼市', '外交部', '接收', '两国', '原则', '各地', '总理', '战略',
         '和平', '框架', '评论', '有序', '装修', '中国', '就业', '友好', '人力资源']],
       dtype='<U9'),
 array([[0.3623712 , 0.36037514, 0.35219163, 0.35109183, 0.3499857 ,
         0.34666985, 0.3426961 , 0.34161803, 0.34010434, 0.3382269 ,
         0.33710504, 0.336056  , 0.33598724, 0.33488944, 0.3303768 ,
         0.32483265, 0.324798  , 0.32201332, 0.3174801 , 0.3153757 ,
         0.3152491 , 0.31338856, 0.31334093, 0.31244045, 0.31202242,
         0.30908576, 0.3086405 , 0.30838227, 0.30605763, 0.3053521 ,
         0.30474398, 0.30268514, 0.30253592, 0.30242488, 0.30227807,
         0.3017046 , 0.30116442, 0.30062813, 0.2996228 , 0.29806197,
         0.2972776 , 0

## 6. search_topics

根据关键词搜索话题，查某词是否属于某话题，属于该主题的概率
search_topics(keywords, num_topics, keywords_neg=None)

- keywords: 关键词列表
- num_topics: 返回话题个数，按照语义相似度从高到低排序
- keywords_neg: 反义词列表

In [14]:
def gen_wordcloud2(query_word, topic_words, word_scores, topic_id, topic_probability):
    """
    query_word: 待查询词
    topic_words: 主题词列表
    word_scores: 主题特征词的权重得分(词语表征主题的能力)
    topic_id: 主题id
    topic_probability: 主题概率
    """
    import pyecharts.options as opts
    from pyecharts.charts import WordCloud
    from IPython.display import display
    
    data = [(word, str(weight)) for word, weight in zip(topic_words, word_scores)]

    wc = WordCloud()
    wc.add(series_name="", data_pair=data, word_size_range=[6, 88])
    title = """Word{query_word}\nTopic_{topic_id}\nProbability:{probability:.2f}""".format(query_word=query_word,
                                                              topic_id=topic_id, 
                                                              probability=topic_probability)
    wc.set_global_opts(
        title_opts=opts.TitleOpts(title=title, 
                                  title_textstyle_opts=opts.TextStyleOpts(font_size=18)),
        tooltip_opts=opts.TooltipOpts(is_show=True))

    display(wc.render_notebook())

In [15]:
query_word = "电影"
topic_wordss, word_scoress, topic_scores, topic_ids = model.search_topics(keywords=[query_word], num_topics=4)
for topic_words, word_scores, topic_score, topic_id in zip(topic_wordss, word_scoress, topic_scores, topic_ids):
    if topic_score>0.5:
        gen_wordcloud2(query_word=query_word, 
                       topic_words=topic_words, 
                       word_scores=word_scores, 
                       topic_id=topic_id, topic_probability=topic_score)

## 7. query_topics

根据一段文本寻找最符合该文本的话题
query_topics(query, num_topics)

- query: 查询文本，注意是用空格间隔词语的文本
- num_topics: 返回的话题数

返回话题特征词列表， 话题特征词权重， 话题概率， 话题id

In [16]:
querytext = '刘晓庆 55 岁 近日 颁奖礼 刘晓庆 一袭 宝蓝色 超低 胸 V 领 长裙 亮相 轻薄 蕾丝 奢华 皮草 艳丽 色彩 翠绿'
topic_words, word_scores, topic_scores, topic_ids = model.query_topics(query=querytext, 
                                                                       num_topics=2)
print('可能归属的话题有: ', topic_ids)
print('归属于该话题的概率', topic_scores)

可能归属的话题有:  [1 4]
归属于该话题的概率 [0.32036728 0.1276904 ]


## 8. search_documents_by_keywords

根据关键词，筛选文档

search_documents_by_keywords(keywords, 
                             num_docs, 
                             keywords_neg=None, 
                             return_documents=True)



In [17]:
#文档， 语义相关性， 文档id
docs, scores, doc_ids = model.search_documents_by_keywords(keywords=['搭配'], 
                                                         num_docs=3, 
                                                         keywords_neg=None, 
                                                         return_documents=True)
for doc, score, doc_id in zip(docs, scores, doc_ids):
    print(f"Document: {doc_id}, Semantic similarity: {score}")
    print(doc)
    print('----------')
    print()

Document: 106, Semantic similarity: 0.4943176805973053
白色 短裙 百变 休闲 感 要点 一定 敞开 衬衫 配合 牛仔裤 休闲 感 短裤 衬衫 短 敞开 显得 好好 穿 裤子 搭配 七分裤 遮住 臀部 长度 关键 尽量 选择 艳丽 颜色 带 出 青春 感 NO.3     白色 短裙 tips :   白色 短裙 + 粉色 上衣 这是 一套 减龄 百分百 搭配 白色 短裙 本来 清纯 粉色 上衣 搭配 更加 具有活力 tips :   白色 短裙 + 抹胸 + 外套 想要 性感 一点 就加 一件 抹胸 抹胸 胸前 构造 曲线 完美 再加 外套 保暖 得体 看似 简单 一款 搭配 其实 暗地里 偷偷地 修饰 身材
----------

Document: 870, Semantic similarity: 0.4483542740345001
组图 看达人 演绎 豹纹 军装 风 导语 懂得 潮流 总是 知道 适合 今冬 流行 亮点 太 军装 豹纹 类似 民族风情 想要 知道 搭配 快 看看 时尚 达 穿 军绿色 宽松 款 大衣 不失 俏皮 味道 高腰 设计 短裙 有效 提升 腰线 衬托出 修长 美腿 豹纹 今年 冬季 抢眼 搭配 元素 加上 驼色 针织衫 灰色 围巾 暖 棕色 手 挎包 整体 色调 统一 迷人 棕色 蓝色 结合能 眼前一亮 简洁 款式 依然 突显 独特 品味 宽松 针织 外套 衬托出 优美 身形 搭配 同样 沉闷 黑色 包包 性感 丝袜 装扮 依然 透露 出 迷人 气息 立领 衬衫 加上 深黄 高腰 裤 摩登 感 十足 随意 披上 外套 更显 慵懒 个性 法式 风情
----------

Document: 450, Semantic similarity: 0.4471719563007355
街 拍 爱 招摇过市 毛茸茸 ( 组图 ) 导语 皮草 每个 冬天 可能 丢弃 每个 需要 温暖 早些 相比 人造皮 草比 真皮 草 风头 更劲 时尚 环保 大牌 秀 场上 超模 一个个 穿着 人造皮 草 “ 招摇过市 ” 之后 街头 潮人 没有 理由 拒绝 外形 酷酷 这件 气场 皮草 单品 配合默契 摇滚 风 配饰 搭配 黑色 皮草 长 背心 更显 利落 酷酷 黑色 皮草 搭配 蓝色 衬衣 不同 感

## 9. search_documents_by_topic

根据指定的topic_id， 显示该主题前num_docs个文档，显示的文档是根据概率从高到低降序显示

In [18]:
#查看topic4的前5条文档
topic_id = 4
documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=topic_id, num_docs=5)
for doc, score, doc_id in zip(documents, document_scores, document_ids):
    print(f"Document: {doc_id}, Semantic similarity: {score}")
    print("-----------")
    print(doc)
    print("-----------")
    print()

Document: 905, Semantic similarity: 0.4941929578781128
-----------
现场 阿联 第三节 未 亮相   奇才 连续 3 记 重扣 逆转 比分 新浪 体育讯 北京 时间 4 2 奇才 主场 迎战 联盟 垫底 骑士 奇才 新秀 后卫 约翰 - 沃尔因 一场 对阵 热火 比赛 斗殴 禁赛 一场 伤愈 复出 安德雷 - 布 莱切 回到 首发 阵容 奇才 本赛季 首场 客场 胜利 面对 骑士 取得 当时 奇才 115 - 110 击败 对手 上半场 骑士 命中率 达到 53.8% 奇才 仅 44% 骑士 希克 森 ( 16 ) 塞 申斯 ( 12 ) 得分 双 奇才 布 莱切 ( 22 ) 麦基 ( 12 ) 埃文斯 4 投 0 仅 抢下 1 篮板 易建联 上场 7 08 2 投 0 抢下 3 篮板 异地 再战 埃文斯 终于 得分 抢断 吉后 犯规 两罚 命中 吉 随即 突破 上篮 命中 回敬 球 杰 弗斯 三分 不进 吉 抢下 篮板 上篮 再进 一球 布 莱切 中投 命中 霍林斯 篮下 出手 不进 布 莱切 抢下 篮板 此后 克劳福德 连续 突破 先是 助攻 麦基 扣篮 盖掉 戴维斯 投篮 助攻 布 莱切 扣篮 戴维斯 运球 被断 布 莱切 传给 杰 弗斯 一记 暴扣 奇才 连续 3 次 颇具 气势 扣篮 连得 6 反超 骑士 1 骑士 请求 暂停 回到 比赛 吉 上篮 不进 麦基 低位 单打 不进 布 莱切 抢下 篮板 3 得手 骑士 进攻 24 违例 奇才 越战越勇 克劳福德 身体 失去 重心 情况 仍然 将球 投进 一个打 3 骑士 连续 吉 挺身而出 三分 命中 个人 已经 得到 10 此人 本赛季 短暂 效力 奇才 麦基 中投 不进 布 莱切 抢下 前场 篮板 将球 放进 麦基 防守 领到 犯规 希克 森两罚 命中 麦基 强攻 造成 霍林斯 犯规 两罚 一中 戴维斯 三分 不进 克劳福德 跑 投 命中 戴维斯 突分 霍林斯 暴扣 命中 回过头来 克劳福德 助攻 麦基扣 劲 爆 哈兰 高迪 中投 不进 克劳福德 投篮 偏出 布 莱切 3 报价 连续 抢 篮板 进攻 最后 犯规 两罚 一中 现在 已经 得到 32 18 篮板 布 莱切 底线 遭 报价 分球 埃文斯 三分 命中 霍林斯 篮下 重扣 奇才 请

## 10. get_topic_hierarchy
对话题进行分类，需要
1. 先执行model.hierarchical_topic_reduction
2. 再执行model.get_topic_hierarchy。



In [19]:
# 将话题分为2类
model.hierarchical_topic_reduction(num_topics=2)
model.get_topic_hierarchy()

[[7, 6, 1, 8, 5, 4, 3], [2, 0]]

## 11. similar_words
查找相似词， 该方法其实也可以用于扩充词典。


similar_words(keywords, num_words, keywords_neg=None)
- keywords: 待查询关键词列表
- num_words: 返回相似词个数
- keywords_neg: 指定反义词列表




In [20]:
# 查找【增进】的最相似的10个词
model.similar_words(keywords=["增进"], 
                    num_words=10, 
                    keywords_neg=None)

(array(['两国关系', '两国', '温家宝', '王刚', '战略', '友好', '中欧', '政治', '会见', '人民'],
       dtype='<U4'),
 array([0.50498132, 0.49835259, 0.4636392 , 0.45802986, 0.45299921,
        0.44836198, 0.43550295, 0.43471974, 0.43099192, 0.42711113]))